In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils.data_utils import get_file
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
#import cpickle as pickle
#import bcolz
import re
from numpy.random import random, permutation, randn, normal, uniform, choice

In [12]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print(len(text))

614400/600901 [==============================] - 0s 0us/step
600893


In [21]:

chars = sorted(list(set(text)))
print(len(chars)+1)

85


In [20]:
chars=chars.insert(0, '\0')

In [22]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

In [23]:
total_index = [char_to_index[char] for char in text]

In [25]:
pred_num = 25
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [26]:

X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [28]:
X

[array([39, 43, 57, ..., 57, 56, 53]),
 array([41, 70, 66, ...,  1, 56, 66]),
 array([28, 73, 23, ..., 61, 60, 55]),
 array([29, 72,  1, ..., 66, 61, 72]),
 array([24, 60, 32, ..., 71, 71, 61]),
 array([26,  1, 71, ..., 61, 72, 72]),
 array([28, 61,  1, ..., 71, 71, 77]),
 array([ 0, 71, 72, ..., 72,  1,  7]),
 array([ 0,  1, 60, ..., 57, 53,  1]),
 array([ 0, 53, 57, ..., 56, 71, 62]),
 array([42,  1, 70, ...,  1,  1, 73]),
 array([44, 75, 57, ..., 73, 57, 71]),
 array([39, 67,  1, ..., 68, 71, 72]),
 array([39, 65, 66, ..., 67, 71,  1]),
 array([38, 53, 67, ..., 66, 57, 53]),
 array([42, 66, 72, ...,  1, 66, 71]),
 array([32,  8,  1, ..., 54, 72,  1]),
 array([37,  8, 59, ..., 77, 61, 72]),
 array([30, 75, 70, ...,  1, 53, 60]),
 array([ 1, 60, 67, ..., 72, 64, 57]),
 array([72, 53, 73, ..., 60,  1, 77]),
 array([60, 72, 66, ..., 57, 72,  1]),
 array([53,  1, 56, ...,  0, 67, 75]),
 array([72, 72,  0, ..., 25,  1, 57]),
 array([ 1, 60, 58, ..., 73, 71, 70])]

(24033,)

In [31]:

hidden_layers = 256
vocab_size = 85
n_fac = 42

In [46]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=25),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [47]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 25, 42)            3570      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 256)               76544     
                                                                 
 dense_3 (Dense)             (None, 85)                21845     
                                                                 
Total params: 101,959
Trainable params: 101,959
Non-trainable params: 0
_________________________________________________________________


In [48]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

array([[39, 41, 28, ..., 53, 72,  1],
       [43, 70, 73, ...,  1, 72, 60],
       [57, 66, 23, ..., 56,  0, 58],
       ...,
       [57,  1, 61, ...,  0, 25, 73],
       [56, 56, 60, ..., 67,  1, 71],
       [53, 66, 55, ..., 75, 57, 70]])

In [49]:

model.fit(np.stack(X, 1), Y, batch_size=64, epochs=5)

Epoch 1/5
376/376 [==============================] - 18s 47ms/step - loss: 3.2302
Epoch 2/5
376/376 [==============================] - 18s 47ms/step - loss: 2.6503
Epoch 3/5
376/376 [==============================] - 18s 47ms/step - loss: 2.4061
Epoch 4/5
376/376 [==============================] - 18s 47ms/step - loss: 2.2613
Epoch 5/5
376/376 [==============================] - 17s 46ms/step - loss: 2.1502


In [39]:
def predict_next_char(inp):
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]


In [60]:
predict_next_char('those w')

'h'

In [61]:
predict_next_char(' th')

'e'